
# 📚 ComfyUI Scene Generator for Books  
Generate high‑quality image sequences from a reference picture using **Stable Diffusion XL (SDXL)** on **ComfyUI**, all without ever leaving this Google Colab notebook.

---

### What you’ll get
| | |
|---|---|
|📷 **Upload an image** to use as the starting frame|🎨 **Style box** – describe the mood (e.g. “dark anime”, “vintage illustration”, etc.)|
|🔢 **Images slider** – choose **1 – 100** renders per scene|🔁 **Soft‑variation toggle** to introduce gentle pose / lighting / background changes|
|👁️ **Instant preview** of every frame inside the notebook|✅ **Approve & save** only the frames you like → auto‑copies to Google Drive|

> **Tip:** The notebook also installs the full ComfyUI web interface – open it in another browser tab if you want node‑level control.


In [ ]:

#@title 🔧 Install ComfyUI & dependencies ↘︎ (takes 2‑3 min)
!pip -q install --upgrade diffusers[torch] transformers accelerate safetensors einops xformers ipywidgets nbformat
!git clone --depth 1 https://github.com/comfyanonymous/ComfyUI.git || echo "ComfyUI already cloned"


In [ ]:

#@title 🔗 Mount Google Drive (for saving approved frames)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
OUTPUT_DIR = '/content/drive/MyDrive/ComfyUI_Generated'
import os, pathlib
pathlib.Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
print(f"Images you approve will be copied to: {OUTPUT_DIR}")


In [ ]:

#@title ▶️ (Optional) Launch the ComfyUI web interface
# Skips if already running. Access on  https://<colab‑host>:8188  (use Colab ↗ menu → ‘Preview’)
import subprocess, os, signal, time, textwrap, json, pathlib, random, io, base64, types, sys, math, functools
if not os.path.exists('/content/ComfyUI'):
    print("ComfyUI folder missing – re‑run install cell")
else:
    proc = subprocess.Popen(['python', 'main.py', '--listen', '0.0.0.0', '--port', '8188', '--disable-auto-launch'], cwd='/content/ComfyUI')
    time.sleep(5)
    print("ComfyUI server started on http://localhost:8188 (use 'Open in new tab' from Colab)")


In [ ]:

#@title 🚀 Load SDXL Img2Img pipeline (≈ 1 min on first run)
import torch, gc, contextlib, warnings
from diffusers import StableDiffusionXLImg2ImgPipeline

warnings.filterwarnings("ignore")
pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    variant="fp16",
    torch_dtype=torch.float16,
).to("cuda")
pipe.enable_xformers_memory_efficient_attention()
print("✔ SDXL pipeline ready")


In [ ]:

#@title 🎛️ Interactive Generator
import ipywidgets as wd, io, random, math
from PIL import Image
from IPython.display import display, clear_output

uploader       = wd.FileUpload(accept='image/*', multiple=False, description='Upload ↗')
style_txt      = wd.Text(value='', placeholder='e.g. dark anime', description='Style:')
num_slider     = wd.IntSlider(value=10, min=1, max=100, step=1, description='Images:', continuous_update=False)
variation_ck   = wd.Checkbox(value=True, description='Soft variation')
generate_btn   = wd.Button(description='Generate', button_style='success')
approve_btn    = wd.Button(description='Save selected', button_style='primary')
preview_out    = wd.Output(layout={'border': '1px solid #ddd','padding':'4px'})
select_multi   = wd.SelectMultiple(options=[], description='Pick ⬇')

last_images = []  # holds PIL images in current batch

def _pil_to_bytes(img):
    bio = io.BytesIO()
    img.save(bio, format='PNG')
    return bio.getvalue()

def on_generate(b):
    with preview_out:
        clear_output(wait=True)
        if len(uploader.value) == 0:
            print("⬆️ Please upload a reference image first."); return
        # prepare init img
        ref_bytes = next(iter(uploader.value.values()))['content']
        init_img = Image.open(io.BytesIO(ref_bytes)).convert('RGB')
        init_img = init_img.resize((1024, 1024))
        global last_images
        last_images = []
        options = []
        base_prompt = style_txt.value.strip() or "illustration"
        print("⏳ Generating…")
        for i in range(num_slider.value):
            prompt = base_prompt
            if variation_ck.value:
                prompt += ", " + random.choice(["alternate angle", "dynamic lighting", "subtle pose shift", "different background"])
            image = pipe(
                prompt            = prompt,
                image             = init_img,
                strength          = 0.7,
                guidance_scale    = 7.0,
                num_inference_steps=30,
            ).images[0]
            last_images.append(image)
            options.append((f"Image {i+1}", i))
            # display thumbnail grid
        thumbs = [wd.Image(value=_pil_to_bytes(im), format='png', width=180) for im in last_images]
        rows = [wd.HBox(thumbs[i:i+4]) for i in range(0, len(thumbs), 4)]
        display(wd.VBox(rows))
        select_multi.options = options
        print("✔ Generation complete – select frames to save")

def on_save(b):
    if not select_multi.value:
        with preview_out: print("🔔 No frames selected.")
        return
    count = 0
    for idx in select_multi.value:
        fname = f"{OUTPUT_DIR}/scene_{idx:03d}.png"
        last_images[idx].save(fname)
        count += 1
    with preview_out:
        print(f"✅ Saved {count} images → {OUTPUT_DIR}")

generate_btn.on_click(on_generate)
approve_btn.on_click(on_save)

ui = wd.VBox([wd.HBox([uploader, style_txt]), num_slider, variation_ck,
              wd.HBox([generate_btn, approve_btn]), select_multi, preview_out])
display(ui)
